In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime,gc,math
import random
%matplotlib inline
import gc
import lightgbm as lgb
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold

In [2]:
def active_time_transform(df):
    df['role_created_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
    df['role_created_active_time'] = df['role_created_active_time'].apply(lambda x:str(x)[1:-1])
    temp = df['role_created_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                        3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
    for i in range(len(temp.columns)):
        temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
    df = df.join(temp).drop(['role_created_active_time'], axis=1)
    return df

def pay_grade_transform(df):
    df['pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
    df['pay_grade'] = df['pay_grade'].apply(lambda x:str(x)[1:-1])
    temp = df['pay_grade'].str.split(',',expand=True).rename(columns={0:'pay_grade_1',
                1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'})
    df = df.join(temp).drop(['pay_grade'], axis=1)
    return df

In [3]:
%%time
role_info = pd.read_csv('./data/mr_role_6d.csv')
role_info.drop_duplicates(subset=['user_id','cp_server_no','cp_role_id'],inplace=True)
role_info = active_time_transform(role_info)
role_info = pay_grade_transform(role_info)

Wall time: 2min 4s


In [4]:
# 删除2020-7-19日前30天的数据
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'], format='%Y-%m-%d %H:%M:%S')
role_info.drop(role_info[role_info['create_role_time'] > (role_info['create_role_time'].max() + 
                                                          datetime.timedelta(-30))].index,inplace=True)

In [5]:
# 删除没有心跳数据的样本(1、没有登录;2、没有拿到心跳)
role_info.dropna(subset=['role_created_login_num'], inplace=True)
role_info.dropna(subset=['role_created_online'], inplace=True)
role_info['pay_num'].fillna(0, inplace=True)
role_info['pay_sum'].fillna(0, inplace=True)
role_info['role_created_active'] = role_info['role_created_active'].clip(0,6)
role_info['pay_rate'] = role_info['pay_num'] / (role_info['role_created_active'] + 1e-4)
role_info['pay_avg'] = role_info['pay_sum'] / (role_info['pay_num'] + 1e-4)

In [6]:
role_created_30_pay_sum = pd.read_csv('./data/role_created_30_pay_sum.csv',index_col=0)
# 对连接的关键字进行字符化或者整型化的调整
role_created_30_pay_sum['user_id'] = role_created_30_pay_sum['user_id'].astype(str)
role_created_30_pay_sum['mgame_id'] = role_created_30_pay_sum['mgame_id'].astype(str)
role_created_30_pay_sum['cp_server_no'] = role_created_30_pay_sum['cp_server_no'].astype(str)
role_created_30_pay_sum['cp_role_id'] = role_created_30_pay_sum['cp_role_id'].astype(str)
role_info['user_id'] = role_info['user_id'].astype(str)
role_info['mgame_id'] = role_info['mgame_id'].astype(str)
role_info['cp_server_no'] = role_info['cp_server_no'].astype(str)
role_info['cp_role_id'] = role_info['cp_role_id'].astype(str)
role_created_30_pay_sum.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info = pd.merge(role_info,role_created_30_pay_sum,on=['user_id','mgame_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [7]:
select_features = ['user_id','cp_server_no','cp_role_id','role_created_login_num',
       'role_created_active', 'role_created_online',
       'max_role_level', 'ip_num',
       'pay_num', 'pay_sum', 'active_0-8', 'active_8-12', 'active_12-14',
       'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
       'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
       'pay_rate', 'pay_avg','role_created_30_pay_sum']
role_info = role_info[select_features]
role_info['role_created_30_pay_sum'].fillna(0,inplace=True)
col_list = ['active_0-8','active_8-12','active_12-14','active_14-18','active_18-24','pay_grade_1','pay_grade_2',
            'pay_grade_3','pay_grade_4','pay_grade_5','pay_grade_6']
for col in col_list:
    role_info[col] = pd.to_numeric(role_info[col], errors='coerce')

In [8]:
role_info.to_pickle('./role_info_6d.pkl')

In [9]:
df_train, df_test = train_test_split(role_info,test_size=0.3)
df_train_pay = df_train[df_train['pay_sum']>0]
df_train_nopay = df_train[df_train['pay_sum']==0]

In [10]:
target = df_train_pay['role_created_30_pay_sum']
features = df_train_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
target_ln = np.log1p(target)
X_val, x_test, Y_val, y_test = train_test_split(features,target_ln, test_size=0.3)

In [11]:
# lgb的调参与交叉验证
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# 设置参数初始值，不含交叉验证参数
print('设置参数')
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'nthread':4,
    'learning_rate':0.05
}

# 交叉验证（调参）
print('交叉验证')
min_rmse = float('5')
best_params = {}

# 准确率
print('调参1：提高准确率')
for num_leaves in range(5, 100, 5):
    for max_depth in range(3, 8, 1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50     
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

if 'num_leaves' and 'max_depth' in best_params.keys():
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']

# 过拟合
print('调参2：降低过拟合')
min_rmse = float('5')
for max_bin in range(5, 256, 10):
    for min_data_in_leaf in range(1, 102, 10):
        params['max_bin'] = max_bin
        params['min_data_in_leaf'] = min_data_in_leaf
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50 
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['min_data_in_leaf'] = min_data_in_leaf
            best_params['max_bin'] = max_bin
            
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']

print('调参3：降低过拟合')
min_rmse = float('5')
for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq

            cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
            mean_rmse = pd.Series(cv_results['rmse-mean']).min()
            boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
            if mean_rmse <= min_rmse:
                min_rmse = mean_rmse
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']

print('调参4：降低过拟合')
min_rmse = float('5')
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']

print("调参5：降低过拟合2")
min_rmse = float('5')
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50    
                            )
    mean_rmse = pd.Series(cv_results['rmse-mean']).min()
    boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
    if mean_rmse >= min_rmse:
        min_rmse = mean_rmse
        
    best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
print(best_params)

设置参数
交叉验证
调参1：提高准确率
[50]	cv_agg's rmse: 0.775593 + 0.00672546
[100]	cv_agg's rmse: 0.748547 + 0.0067596
[50]	cv_agg's rmse: 0.775593 + 0.00672546
[100]	cv_agg's rmse: 0.74849 + 0.00675172
[50]	cv_agg's rmse: 0.775593 + 0.00672546
[100]	cv_agg's rmse: 0.74849 + 0.00675172
[50]	cv_agg's rmse: 0.775593 + 0.00672546
[100]	cv_agg's rmse: 0.74849 + 0.00675172
[50]	cv_agg's rmse: 0.775593 + 0.00672546
[100]	cv_agg's rmse: 0.74849 + 0.00675172
[50]	cv_agg's rmse: 0.76593 + 0.00680032
[100]	cv_agg's rmse: 0.74724 + 0.00656598
[50]	cv_agg's rmse: 0.763342 + 0.00697047
[100]	cv_agg's rmse: 0.746749 + 0.00659061
[50]	cv_agg's rmse: 0.763338 + 0.0069603
[100]	cv_agg's rmse: 0.746621 + 0.00658153
[50]	cv_agg's rmse: 0.763338 + 0.0069603
[100]	cv_agg's rmse: 0.74657 + 0.00665129
[50]	cv_agg's rmse: 0.763338 + 0.0069603
[100]	cv_agg's rmse: 0.746582 + 0.00666896
[50]	cv_agg's rmse: 0.76593 + 0.00680032
[100]	cv_agg's rmse: 0.74724 + 0.00656598
[50]	cv_agg's rmse: 0.760552 + 0.0069751
[100]	cv_agg's rm

[50]	cv_agg's rmse: 0.759266 + 0.00707244
[100]	cv_agg's rmse: 0.746168 + 0.00656352
[50]	cv_agg's rmse: 0.759193 + 0.00701402
[100]	cv_agg's rmse: 0.746099 + 0.00660847
[50]	cv_agg's rmse: 0.759178 + 0.00697152
[100]	cv_agg's rmse: 0.746089 + 0.0065581
[50]	cv_agg's rmse: 0.759192 + 0.00700575
[100]	cv_agg's rmse: 0.745967 + 0.00664461
[50]	cv_agg's rmse: 0.759146 + 0.00696492
[100]	cv_agg's rmse: 0.746035 + 0.00664589
[50]	cv_agg's rmse: 0.759206 + 0.00695786
[100]	cv_agg's rmse: 0.746015 + 0.00665104
[50]	cv_agg's rmse: 0.759224 + 0.00695666
[100]	cv_agg's rmse: 0.746082 + 0.00661078
[50]	cv_agg's rmse: 0.759222 + 0.00701284
[100]	cv_agg's rmse: 0.746049 + 0.00665844
[50]	cv_agg's rmse: 0.759195 + 0.00696472
[100]	cv_agg's rmse: 0.745958 + 0.0066058
[50]	cv_agg's rmse: 0.759338 + 0.00716983
[100]	cv_agg's rmse: 0.746075 + 0.00658942
[50]	cv_agg's rmse: 0.759288 + 0.00713272
[100]	cv_agg's rmse: 0.746246 + 0.00667426
[50]	cv_agg's rmse: 0.759266 + 0.00707244
[100]	cv_agg's rmse: 0.74

[50]	cv_agg's rmse: 0.759338 + 0.00716983
[100]	cv_agg's rmse: 0.746075 + 0.00658942
[50]	cv_agg's rmse: 0.759288 + 0.00713272
[100]	cv_agg's rmse: 0.746246 + 0.00667426
[50]	cv_agg's rmse: 0.759266 + 0.00707244
[100]	cv_agg's rmse: 0.746168 + 0.00656352
[50]	cv_agg's rmse: 0.759193 + 0.00701402
[100]	cv_agg's rmse: 0.746099 + 0.00660847
[50]	cv_agg's rmse: 0.759178 + 0.00697152
[100]	cv_agg's rmse: 0.746089 + 0.0065581
[50]	cv_agg's rmse: 0.759192 + 0.00700575
[100]	cv_agg's rmse: 0.745967 + 0.00664461
[50]	cv_agg's rmse: 0.759146 + 0.00696492
[100]	cv_agg's rmse: 0.746035 + 0.00664589
[50]	cv_agg's rmse: 0.759206 + 0.00695786
[100]	cv_agg's rmse: 0.746015 + 0.00665104
[50]	cv_agg's rmse: 0.759224 + 0.00695666
[100]	cv_agg's rmse: 0.746082 + 0.00661078
[50]	cv_agg's rmse: 0.759222 + 0.00701284
[100]	cv_agg's rmse: 0.746049 + 0.00665844
[50]	cv_agg's rmse: 0.759195 + 0.00696472
[100]	cv_agg's rmse: 0.745958 + 0.0066058
[50]	cv_agg's rmse: 0.759338 + 0.00716983
[100]	cv_agg's rmse: 0.74

[50]	cv_agg's rmse: 0.759222 + 0.00701284
[100]	cv_agg's rmse: 0.746049 + 0.00665844
[50]	cv_agg's rmse: 0.759195 + 0.00696472
[100]	cv_agg's rmse: 0.745958 + 0.0066058
[50]	cv_agg's rmse: 0.759338 + 0.00716983
[100]	cv_agg's rmse: 0.746075 + 0.00658942
[50]	cv_agg's rmse: 0.759288 + 0.00713272
[100]	cv_agg's rmse: 0.746246 + 0.00667426
[50]	cv_agg's rmse: 0.759266 + 0.00707244
[100]	cv_agg's rmse: 0.746168 + 0.00656352
[50]	cv_agg's rmse: 0.759193 + 0.00701402
[100]	cv_agg's rmse: 0.746099 + 0.00660847
[50]	cv_agg's rmse: 0.759178 + 0.00697152
[100]	cv_agg's rmse: 0.746089 + 0.0065581
[50]	cv_agg's rmse: 0.759192 + 0.00700575
[100]	cv_agg's rmse: 0.745967 + 0.00664461
[50]	cv_agg's rmse: 0.759146 + 0.00696492
[100]	cv_agg's rmse: 0.746035 + 0.00664589
[50]	cv_agg's rmse: 0.759206 + 0.00695786
[100]	cv_agg's rmse: 0.746015 + 0.00665104
[50]	cv_agg's rmse: 0.759224 + 0.00695666
[100]	cv_agg's rmse: 0.746082 + 0.00661078
[50]	cv_agg's rmse: 0.759222 + 0.00701284
[100]	cv_agg's rmse: 0.74

[100]	cv_agg's rmse: 0.747803 + 0.00633324
[50]	cv_agg's rmse: 0.761784 + 0.00694116
[100]	cv_agg's rmse: 0.748154 + 0.00637515
[50]	cv_agg's rmse: 0.762227 + 0.00691135
[100]	cv_agg's rmse: 0.747981 + 0.00605378
[50]	cv_agg's rmse: 0.762383 + 0.00675603
[100]	cv_agg's rmse: 0.74793 + 0.00625153
[50]	cv_agg's rmse: 0.761763 + 0.00703855
[100]	cv_agg's rmse: 0.746788 + 0.00677204
[50]	cv_agg's rmse: 0.761636 + 0.00681398
[100]	cv_agg's rmse: 0.74687 + 0.00666641
[50]	cv_agg's rmse: 0.761893 + 0.00686745
[100]	cv_agg's rmse: 0.747142 + 0.00656578
[50]	cv_agg's rmse: 0.762013 + 0.0063803
[100]	cv_agg's rmse: 0.747253 + 0.00629632
[50]	cv_agg's rmse: 0.761853 + 0.00698656
[100]	cv_agg's rmse: 0.747318 + 0.00639291
[50]	cv_agg's rmse: 0.761769 + 0.00627572
[100]	cv_agg's rmse: 0.747044 + 0.00619203
[50]	cv_agg's rmse: 0.762236 + 0.00637892
[100]	cv_agg's rmse: 0.747528 + 0.00644784
[50]	cv_agg's rmse: 0.761618 + 0.00699779
[100]	cv_agg's rmse: 0.747278 + 0.00655649
[50]	cv_agg's rmse: 0.761

[100]	cv_agg's rmse: 0.74646 + 0.00638113
[50]	cv_agg's rmse: 0.760721 + 0.00686515
[100]	cv_agg's rmse: 0.74637 + 0.00669873
[50]	cv_agg's rmse: 0.760441 + 0.00685794
[100]	cv_agg's rmse: 0.746293 + 0.0064484
[50]	cv_agg's rmse: 0.760587 + 0.00682334
[100]	cv_agg's rmse: 0.746337 + 0.00656095
[50]	cv_agg's rmse: 0.760769 + 0.00677384
[100]	cv_agg's rmse: 0.746371 + 0.00660307
[50]	cv_agg's rmse: 0.760717 + 0.00683172
[100]	cv_agg's rmse: 0.746465 + 0.0065332
[50]	cv_agg's rmse: 0.760668 + 0.00684005
[100]	cv_agg's rmse: 0.746523 + 0.00651467
[50]	cv_agg's rmse: 0.760731 + 0.00690767
[100]	cv_agg's rmse: 0.746368 + 0.00665746
[50]	cv_agg's rmse: 0.760731 + 0.00690767
[100]	cv_agg's rmse: 0.746368 + 0.00665746
[50]	cv_agg's rmse: 0.760731 + 0.00690767
[100]	cv_agg's rmse: 0.746368 + 0.00665746
[50]	cv_agg's rmse: 0.760731 + 0.00690767
[100]	cv_agg's rmse: 0.746368 + 0.00665746
[50]	cv_agg's rmse: 0.760731 + 0.00690767
[100]	cv_agg's rmse: 0.746368 + 0.00665746
[50]	cv_agg's rmse: 0.7607

[50]	cv_agg's rmse: 0.759427 + 0.00697846
[100]	cv_agg's rmse: 0.746053 + 0.00674055
[50]	cv_agg's rmse: 0.759761 + 0.00696339
[100]	cv_agg's rmse: 0.746496 + 0.0065355
[50]	cv_agg's rmse: 0.760051 + 0.00678923
[100]	cv_agg's rmse: 0.746772 + 0.00636062
[50]	cv_agg's rmse: 0.759981 + 0.0070055
[100]	cv_agg's rmse: 0.74667 + 0.00649884
[50]	cv_agg's rmse: 0.759569 + 0.00669752
[100]	cv_agg's rmse: 0.746269 + 0.00639382
[50]	cv_agg's rmse: 0.759926 + 0.00690719
[100]	cv_agg's rmse: 0.746634 + 0.00667576
[50]	cv_agg's rmse: 0.759798 + 0.00719429
[100]	cv_agg's rmse: 0.746563 + 0.00665033
[50]	cv_agg's rmse: 0.760102 + 0.00695955
[100]	cv_agg's rmse: 0.746666 + 0.00636786
[50]	cv_agg's rmse: 0.759986 + 0.00685408
[100]	cv_agg's rmse: 0.746609 + 0.00626625
[50]	cv_agg's rmse: 0.759689 + 0.0069321
[100]	cv_agg's rmse: 0.746147 + 0.00659555
[50]	cv_agg's rmse: 0.759679 + 0.00698426
[100]	cv_agg's rmse: 0.746042 + 0.00659209
[50]	cv_agg's rmse: 0.759915 + 0.00707359
[100]	cv_agg's rmse: 0.7460

[50]	cv_agg's rmse: 0.759436 + 0.00699294
[100]	cv_agg's rmse: 0.746147 + 0.00662715
[50]	cv_agg's rmse: 0.759436 + 0.00699294
[100]	cv_agg's rmse: 0.746147 + 0.00662715
[50]	cv_agg's rmse: 0.759195 + 0.00696472
[100]	cv_agg's rmse: 0.745958 + 0.0066058
[50]	cv_agg's rmse: 0.759269 + 0.00659302
[100]	cv_agg's rmse: 0.746636 + 0.00655964
[50]	cv_agg's rmse: 0.759633 + 0.00679544
[100]	cv_agg's rmse: 0.746823 + 0.00648718
[50]	cv_agg's rmse: 0.75989 + 0.00630966
[100]	cv_agg's rmse: 0.747176 + 0.00634706
[50]	cv_agg's rmse: 0.759629 + 0.00710727
[100]	cv_agg's rmse: 0.747101 + 0.00658556
[50]	cv_agg's rmse: 0.759817 + 0.00649213
[100]	cv_agg's rmse: 0.747444 + 0.00625098
[50]	cv_agg's rmse: 0.760103 + 0.00646814
[100]	cv_agg's rmse: 0.746907 + 0.00675137
[50]	cv_agg's rmse: 0.759344 + 0.00730799
[100]	cv_agg's rmse: 0.747489 + 0.00683309
[50]	cv_agg's rmse: 0.75979 + 0.00720736
[100]	cv_agg's rmse: 0.74753 + 0.00632466
[50]	cv_agg's rmse: 0.759865 + 0.00708438
[100]	cv_agg's rmse: 0.7471

[50]	cv_agg's rmse: 0.759234 + 0.00701857
[100]	cv_agg's rmse: 0.745877 + 0.00670457
[50]	cv_agg's rmse: 0.759267 + 0.00698467
[100]	cv_agg's rmse: 0.745936 + 0.00667715
[50]	cv_agg's rmse: 0.75933 + 0.00696302
[100]	cv_agg's rmse: 0.745922 + 0.00666371
[50]	cv_agg's rmse: 0.75931 + 0.0069699
[100]	cv_agg's rmse: 0.74584 + 0.00670762
[50]	cv_agg's rmse: 0.759272 + 0.00698006
[100]	cv_agg's rmse: 0.745867 + 0.00671363
[50]	cv_agg's rmse: 0.759231 + 0.00691914
[100]	cv_agg's rmse: 0.745849 + 0.00659003
[50]	cv_agg's rmse: 0.759231 + 0.00691914
[100]	cv_agg's rmse: 0.745849 + 0.00659003
[50]	cv_agg's rmse: 0.759238 + 0.00690496
[100]	cv_agg's rmse: 0.745875 + 0.00668141
[50]	cv_agg's rmse: 0.759231 + 0.00691914
[100]	cv_agg's rmse: 0.745849 + 0.00659003
[50]	cv_agg's rmse: 0.759238 + 0.00690496
[100]	cv_agg's rmse: 0.745875 + 0.00668141
[50]	cv_agg's rmse: 0.759247 + 0.00695472
[100]	cv_agg's rmse: 0.745919 + 0.00661415
[50]	cv_agg's rmse: 0.759314 + 0.00698993
[100]	cv_agg's rmse: 0.7459

In [12]:
params = {

'boosting_type':'gbdt',
'objective':'regression',
'metric':'rmse',
'nthread':4,
'learning_rate':0.05,
"max_depth":5,
"num_leaves":20,
"max_bin":255,
"min_data_in_leaf":101,
"min_split_gain":1.0,
"feature_fraction": 1,
"bagging_fraction":0.9,
"bagging_freq":10,
"lambda_l1":0.3,
"lambda_l2":0.9,
"metric": "rmse",
}
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_r_cv = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's rmse: 1.72642	valid_1's rmse: 1.72949
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.65738	valid_1's rmse: 1.66037
[3]	training's rmse: 1.59243	valid_1's rmse: 1.59535
[4]	training's rmse: 1.5314	valid_1's rmse: 1.53418
[5]	training's rmse: 1.47413	valid_1's rmse: 1.47692
[6]	training's rmse: 1.42038	valid_1's rmse: 1.42315
[7]	training's rmse: 1.37003	valid_1's rmse: 1.37279
[8]	training's rmse: 1.3229	valid_1's rmse: 1.32567
[9]	training's rmse: 1.2788	valid_1's rmse: 1.28166
[10]	training's rmse: 1.23769	valid_1's rmse: 1.2406
[11]	training's rmse: 1.19927	valid_1's rmse: 1.20224
[12]	training's rmse: 1.16347	valid_1's rmse: 1.16648
[13]	training's rmse: 1.13018	valid_1's rmse: 1.13326
[14]	training's rmse: 1.09921	valid_1's rmse: 1.10239
[15]	training's rmse: 1.07044	valid_1's rmse: 1.07371
[16]	training's rmse: 1.04376	valid_1's rmse: 1.04718
[17]	training's rmse: 1.01904	valid_1's rmse: 1.02256
[18]	training's rmse: 0.996186	va

In [13]:
df_test_pay = df_test[df_test['pay_sum']>0]
df_test_nopay = df_test[df_test['pay_sum']==0]

In [14]:
df_test_part1 = df_test_nopay[['user_id','cp_server_no','cp_role_id','pay_num']].rename(columns=
                                                                                {'pay_num':'predict_30_pay'})

In [15]:
target_test = df_test_pay['role_created_30_pay_sum']
target_test_ln = np.log1p(target_test)
features_test = df_test_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
y_predict = lgb_r_cv.predict(features_test)
y_predict[y_predict<0] = 0
mse = mean_squared_error(np.expm1(y_predict), np.expm1(target_test_ln))
mae = mean_absolute_error(np.expm1(y_predict), np.expm1(target_test_ln))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

2256.2946549404514
5090865.569912851
421.3510900838859


In [16]:
sum(np.expm1(target_test_ln))/sum(np.expm1(y_predict))

1.2977458037075822

In [17]:
df_test_part2 = df_test_pay[['user_id','cp_server_no','cp_role_id']]
df_test_part2['predict_30_pay'] = np.expm1(y_predict) *1.29
pred = df_test_part1.append(df_test_part2)
predict_data = pd.merge(df_test[['user_id','cp_server_no','cp_role_id', 'role_created_30_pay_sum']],pred,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [18]:
mse = mean_squared_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
mae = mean_absolute_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
rmse = mse ** 0.5
print('测试集上的均方根误差:%.2f元'% rmse)
print('测试集上的平均绝对误差:%.2f元'% mae)
print('测试集前30天实际总的付费金额:%.2f元' % predict_data['role_created_30_pay_sum'].sum())
print('测试集前30天预测总的付费金额:%.2f元'% predict_data['predict_30_pay'].sum())
print('预测总金额准确率:',predict_data['predict_30_pay'].sum()/predict_data['role_created_30_pay_sum'].sum())

测试集上的均方根误差:330.96元
测试集上的平均绝对误差:11.19元
测试集前30天实际总的付费金额:29354218.00元
测试集前30天预测总的付费金额:28368592.81元
预测总金额准确率: 0.9664230473752328


In [19]:
import joblib
joblib.dump(lgb_r_cv,'./lgb_r_6d.pkl')

['./lgb_r_6d.pkl']